## Table of Postal Codes for toronto & transform the data into a dataframe

In [56]:
import pandas as pd
import numpy as np
# from IPython.core.display import HTML

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
res = pd.read_html(url)

In [4]:
len(res)

3

In [5]:
torontoPostalCodes = res[0]
torontoPostalCodes.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
torontoPostalCodes.shape

(180, 3)

In [7]:
torontoPostalCodesWithBorough = torontoPostalCodes[torontoPostalCodes['Borough'] != 'Not assigned']
torontoPostalCodesWithBorough.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
torontoPostalCodesWithBorough = torontoPostalCodesWithBorough.reset_index(drop=True)
torontoPostalCodesWithBorough.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
torontoPostalCodesWithBorough.shape

(103, 3)

## Adding Lat & Long to the dataframe

In [10]:
postalCodeCoordinates = pd.read_csv('https://cocl.us/Geospatial_data')
postalCodeCoordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
postalCodeCoordinates.shape

(103, 3)

In [12]:
torontoPostalCodesWithBorough['Latitude'] = np.nan
torontoPostalCodesWithBorough['Longitude'] = np.nan

In [13]:
len1 = len(torontoPostalCodesWithBorough)
len2 = len(postalCodeCoordinates)
for i in range(len1):
    for j in range(len2):
        if (torontoPostalCodesWithBorough.loc[i,'Postal Code'] == postalCodeCoordinates.loc[j, 'Postal Code']):
            torontoPostalCodesWithBorough.loc[i,'Latitude'] = postalCodeCoordinates.loc[j, 'Latitude']
            torontoPostalCodesWithBorough.loc[i,'Longitude'] = postalCodeCoordinates.loc[j, 'Longitude']

In [14]:
torontoPostalCodesWithBorough.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Clustring the neighborhoods

In [15]:
import re
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


#### boroughs that contain the word Toronto

In [16]:
RegEx = 'toronto'
torontoBoroughs = torontoPostalCodesWithBorough[torontoPostalCodesWithBorough['Borough'].str.contains('toronto', case=False, regex=False)]
torontoBoroughs.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [17]:
torontoBoroughs['Borough'].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

In [18]:
torontoBoroughs = torontoBoroughs.reset_index(drop=True)
torontoBoroughs.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [19]:
torontoBoroughs.shape

(39, 5)

# Lat & Long of Toronto

In [20]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto City are {latitude}, {longitude}.')

The geograpical coordinate of Toronto City are 43.65238435, -79.38356765.


# Map of Toronto with neighborhoods superimposed on top

In [59]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(torontoBoroughs['Latitude'], torontoBoroughs['Longitude'], torontoBoroughs['Borough'], torontoBoroughs['Neighbourhood']):
    label = f'{neighborhood}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Define Foursquare Credentials and Version

In [22]:
CLIENT_ID = 'ADMBEOX5DBDY0ABNHHQRWVQ0B3S3Z1FXX0DDBH4X0N5LLWG3' # your Foursquare ID
CLIENT_SECRET = '1TFMWMXQPATEOHFV40CCS5DRF45YO34INLS4UTYZNMIVTD31' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ADMBEOX5DBDY0ABNHHQRWVQ0B3S3Z1FXX0DDBH4X0N5LLWG3
CLIENT_SECRET:1TFMWMXQPATEOHFV40CCS5DRF45YO34INLS4UTYZNMIVTD31


# Explore the first neighbothood in our dataframe

In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

neighborhood_name = torontoBoroughs.loc[0, 'Neighbourhood']
neighborhood_latitude = torontoBoroughs.loc[0, 'Latitude']
neighborhood_longitude = torontoBoroughs.loc[0, 'Longitude']

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={neighborhood_latitude},{neighborhood_longitude}&radius={radius}&limit={LIMIT}'
print(f'the url for {neighborhood_name} neighbourhood is {url}')

the url for Regent Park, Harbourfront neighbourhood is https://api.foursquare.com/v2/venues/explore?&client_id=ADMBEOX5DBDY0ABNHHQRWVQ0B3S3Z1FXX0DDBH4X0N5LLWG3&client_secret=1TFMWMXQPATEOHFV40CCS5DRF45YO34INLS4UTYZNMIVTD31&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100


In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fef67438f29fd1b441969b7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [27]:
nearby_venues.shape

(46, 4)

## Explore neighborhoods in boroughs that contain the word Toronto

#### create a function to repeat the same process to all the neighborhoods in Boroughs that contain Toronto word (torontoBoroughs df)

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius={radius}&limit={LIMIT}'
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
torontoBoroughs_venues = getNearbyVenues(names=torontoBoroughs['Neighbourhood'],
                                   latitudes=torontoBoroughs['Latitude'],
                                   longitudes=torontoBoroughs['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [30]:
print(torontoBoroughs_venues.shape)
torontoBoroughs_venues.head()

(1624, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


#### how many venues were returned for each neighborhood

In [31]:
torontoBoroughs_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,38,38,38,38,38,38


In [39]:
# torontoBoroughs_venues['Venue Category'].unique() # we have Neighborhood category in Venue Category
torontoBoroughs_venues = torontoBoroughs_venues[torontoBoroughs_venues['Venue Category'] != 'Neighborhood']

In [40]:
uniqueCtgsNum = len(torontoBoroughs_venues['Venue Category'].unique())
print(f'There are {uniqueCtgsNum} uniques categories.')

There are 234 uniques categories.


## Analyze Each Neighborhood

In [41]:
# one hot encoding
torontoBoroughs_onehot = pd.get_dummies(torontoBoroughs_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torontoBoroughs_onehot['Neighborhood'] = torontoBoroughs_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torontoBoroughs_onehot.columns[-1]] + list(torontoBoroughs_onehot.columns[:-1])
torontoBoroughs_onehot = torontoBoroughs_onehot[fixed_columns]

torontoBoroughs_onehot.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
torontoBoroughs_onehot.shape

(1620, 235)

#### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [43]:
torontoBoroughs_grouped = torontoBoroughs_onehot.groupby('Neighborhood').mean().reset_index()
torontoBoroughs_grouped

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.015385,0.000000,0.000000,0.015385,0.000000,0.015385
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012658,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012658,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,0.025316
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [44]:
torontoBoroughs_grouped.shape

(39, 235)

#### print each neighborhood along with the top 5 most common venues

In [45]:
num_top_venues = 5

for hood in torontoBoroughs_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = torontoBoroughs_grouped[torontoBoroughs_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.05
2            Beer Bar  0.03
3  Seafood Restaurant  0.03
4         Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4         Stadium  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3          Comic Shop  0.06
4                Park  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0         Coffee Sho

#### put that into a pandas dataframe

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### create the new dataframe and display the top 10 venues for each neighborhood

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torontoBoroughs_grouped['Neighborhood']

for ind in np.arange(torontoBoroughs_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torontoBoroughs_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Bakery,Seafood Restaurant,Cheese Shop,Restaurant,Juice Bar,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Restaurant,Stadium,Intersection,Bar,Bakery,Climbing Gym,Furniture / Home Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Smoke Shop,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Harbor / Marina,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Salad Place,Burger Joint,Thai Restaurant,Ramen Restaurant,Portuguese Restaurant


## Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 5 clusters.

In [48]:
# set number of clusters
kclusters = 5

torontoBoroughs_grouped_clustering = torontoBoroughs_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torontoBoroughs_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 0, 1, 1, 1, 3])

#### create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

torontoBoroughs_merged = torontoBoroughs

# merge toronto_boroughs_grouped with torontoBoroughs dataframe to add latitude/longitude for each neighborhood
torontoBoroughs_merged = torontoBoroughs_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

torontoBoroughs_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Yoga Studio,French Restaurant,Health Food Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,College Auditorium
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Hotel,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Pizza Place,Italian Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Cocktail Bar,American Restaurant,Clothing Store,Gym,Creperie,Moroccan Restaurant,Department Store,Lingerie Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Asian Restaurant,Health Food Store,Coffee Shop,Trail,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant


In [60]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torontoBoroughs_merged['Latitude'], torontoBoroughs_merged['Longitude'], torontoBoroughs_merged['Neighbourhood'], torontoBoroughs_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [51]:
torontoBoroughs_merged.loc[torontoBoroughs_merged['Cluster Labels'] == 0, torontoBoroughs_merged.columns[[1] + list(range(5, torontoBoroughs_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Grocery Store,Café,Park,Candy Store,Baby Store,Restaurant,Italian Restaurant,Nightclub,Bank,Coffee Shop
14,West Toronto,0,Café,Coffee Shop,Breakfast Spot,Restaurant,Stadium,Intersection,Bar,Bakery,Climbing Gym,Furniture / Home Store


#### Cluster 2

In [52]:
torontoBoroughs_merged.loc[torontoBoroughs_merged['Cluster Labels'] == 1, torontoBoroughs_merged.columns[[1] + list(range(5, torontoBoroughs_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Yoga Studio,French Restaurant,Health Food Store
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar,Smoothie Shop,Burrito Place,Sandwich Place,Café,College Auditorium
2,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Hotel,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Pizza Place,Italian Restaurant
3,Downtown Toronto,1,Coffee Shop,Café,Cocktail Bar,American Restaurant,Clothing Store,Gym,Creperie,Moroccan Restaurant,Department Store,Lingerie Store
4,East Toronto,1,Pub,Asian Restaurant,Health Food Store,Coffee Shop,Trail,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant
5,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Bakery,Seafood Restaurant,Cheese Shop,Restaurant,Juice Bar,Hotel
6,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Salad Place,Burger Joint,Thai Restaurant,Ramen Restaurant,Portuguese Restaurant
8,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Deli / Bodega,Gym,Thai Restaurant,Steakhouse,Lounge,Concert Hall
9,West Toronto,1,Pharmacy,Bakery,Pizza Place,Coffee Shop,Brazilian Restaurant,Café,Middle Eastern Restaurant,Supermarket,Bar,Bank
10,Downtown Toronto,1,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Scenic Lookout,Italian Restaurant,Fried Chicken Joint,Brewery,Park


#### Cluster 3

In [53]:
torontoBoroughs_merged.loc[torontoBoroughs_merged['Cluster Labels'] == 2, torontoBoroughs_merged.columns[[1] + list(range(5, torontoBoroughs_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Park,Bus Line,Swim School,Business Service,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
21,Central Toronto,2,Park,Sushi Restaurant,Bus Line,Trail,Jewelry Store,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant
29,Central Toronto,2,Restaurant,Park,Tennis Court,Trail,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop
33,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop


#### Cluster 4

In [54]:
torontoBoroughs_merged.loc[torontoBoroughs_merged['Cluster Labels'] == 3, torontoBoroughs_merged.columns[[1] + list(range(5, torontoBoroughs_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,3,Gym / Fitness Center,Hotel,Park,Dance Studio,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Pizza Place,Gastropub


##### Cluster 5

In [55]:
torontoBoroughs_merged.loc[torontoBoroughs_merged['Cluster Labels'] == 4, torontoBoroughs_merged.columns[[1] + list(range(5, torontoBoroughs_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Music Venue,Ice Cream Shop,Pool,Garden,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio
